## Discharge

#### You will be using the following example code to answer the objectives: 

1. How much water flows into the sink versus discharges at Copperhead and Langle?  

  a. Where does the water get stored (if less)? Where does it come from (if more)?  

  b. How much uncertainty in the measurements? Compare methods and repeat measurements. 

  c. How do our measurements compare to the broader context of discharge records at Langle and Copperhead?  

In [ ]:
#import important packages
import numpy as np
from pandas import read_csv
import matplotlib.pyplot as plt

### Understanding Discharge with Weirs

We can calculate discharge with weirs by using the angle of the v-notch weir with the head of water using the following equation: 

$Q = 4.28*C*tan(\frac{\theta}{2})*(h+k)^{(\frac{5}{2})}$

In [ ]:
#create the weir function
def weir_discharge(head, notch_angle): #notch angle in degrees, head in ft
    """
    Function to calculate discharge from weir measurements
    
    Parameters
    ----------
    
    head : float
        Float containing the head or height of the water in a weir in feet
    
    notch_angle : float
        Float containing the angle of the weir in degrees
        
    Returns
    -------
    
    Q (Discharge) : Float
        Float containing discharge from the weir measurements in cfs

    """
    notch_angle = np.deg2rad(notch_angle)
    C = (0.607165052 - (0.000874466963*notch_angle)  +  ((6.10393334*(10**-6))*notch_angle**2)) #discharge coefficient
    k = 0.0144902648 - 0.00033955535*notch_angle  + 3.29819003*(10**-6)*notch_angle**2  - 1.06215442*(10**-8) *notch_angle**3 #Head Correction Factor
    Q = 4.28 * C * (np.tan(notch_angle/2)) * ((head+ k)**(5/2))
    return Q #this will be in units of cfs!


In [ ]:
#If you would rather have your discharge in L/s rather than cfs, use this function below to convert! 
def liter_conversion(Q_cfs):
    """
      Function to convert discharge from cfs to L/s
    
    Parameters
    ----------
    
    Discharge in cfs : float
        Float containing discharge value in cfs
        
    Returns
    -------
    
    Q (Discharge) : Float
        Float containing converted discharge in L/s

    """
    Q_Ls = Q_cfs*28.31685 #L/s
    return Q_Ls

Let's test our functions below using some example data! 

In [ ]:
#Now use your function by inputting your values! 
discharge_cfs = weir_discharge(1, 90)
print(discharge_cfs, 'cfs') 

In [ ]:
#Or convert this value to L/s
dischargeL = liter_conversion(discharge_cfs)
print(dischargeL, 'L/s')

In [ ]:
# Now calculate discharge for the field measurements using weirs



### Tracers

When we work with tracers, it's important to calibrate our sensors. Check out the data below to see what that looks like!

In [ ]:
#Read calibration data

df_cal = read_csv('data/Example-calibration.csv', 
                index_col=1, 
                parse_dates=True, 
                skiprows=2,                 
                names=['#', 'SpC', 'Temp']
                )

In [ ]:
#Let's look at the data itself
df_cal['5-12-2023 20:27:35':]

This plot below will show what it looks like when we calibrate our data to known concentrations.

In [ ]:
df_cal.SpC.plot()
plt.xlabel('Time')
plt.ylabel('SpC (uS/cm)')
plt.title('Calibration')

Each plateau is a calibration solution, with known concentration of salt, from left to right, they are DI water (0 mg/L), 400 mg/L and 1000 mg/L.

In [ ]:
#Now we are going to look at our calibration curve which known points from the data above
cond = np.array([12, 765, 1997]) #Conductivities with known concentrations! 
C_salt = np.array([0, 400, 1000]) #known concentrations of salt in mg! 

from scipy.stats import linregress
slope, intercept, rvalue, pvalue, stderr = linregress(cond, C_salt)

plt.plot(cond, C_salt, 'o')
x = np.linspace(0,2000,20)
y = slope * x + intercept
plt.plot(x,y)
plt.xlabel('Conductivity')
plt.ylabel('Known concentrations of salt')
plt.title('Best Fit for Salt Calibration')

print('Slope =',slope)

In [ ]:
#To get our conversion factor, we need to calculate k which is the slope from above
k = slope
k

Now that our sensor has been calibrated, we can use the SpC data to find discharge through a breakthrough curve! 

In [ ]:
#Read SpC breakthrough curve data:
Salt_Discharge_df = read_csv('data/Example-Salt-Discharge.csv', 
                index_col=1, 
                parse_dates=True, 
                skiprows=2,                 
              names=['#', 'SpC', 'Temp']
                )
Salt_Discharge_df

In [ ]:
#Let's plot this up to see what we are working with 
Salt_Discharge_df.SpC.plot()
plt.xlabel('Time (sec)')
plt.ylabel('SpC (uS/cm)')
plt.title('SpC Time Series');

We see that we have the entire time series of our data, but we only want our breakthrough curve, you should have the times where you started your measurement
but you can also use the time series to decide where the breakthrough curve exists. Let's trim our data to just the breakthrough curve. 

In [ ]:
#We can first do it visually by plotting
Salt_Discharge_df.SpC['2023-05-11 13:44:30':'2023-05-11 14:30:00'].plot()
plt.xlabel('Time (sec)')
plt.ylabel('SpC (uS/cm)')
plt.title('Breakthrough Curve');

In [ ]:
#Now, we should make this data its own dataframe so we can do some math on it! 
Discharge_BT_Curve = Salt_Discharge_df['2023-05-11 13:44:30':'2023-05-11 14:30:00']
Discharge_BT_Curve

Now we need to do a little math on our data to obtain discharge.

First, correct the SpC so we can compare it to background levels.
We see our SpC jump at around 13:48 so we need to find the mean SpC from before this time, so let's locate where this happens with .loc

In [ ]:
#Now that he have how many rows are collected before we start to see a jump in SpC, let's take the average value
Pre_breakthrough_df = Discharge_BT_Curve[:'2023-05-11 13:48:00'] #choose only rows before our salt kicks in
mean_background_SpC = Pre_breakthrough_df['SpC'].mean() #take the mean SpC of these values
print(mean_background_SpC)

In [ ]:
#Now that we have our background levels, let's make a corrected SpC column in our dataframe 
SpC_corr = Discharge_BT_Curve['SpC'] - mean_background_SpC


In [ ]:
#Let's convert our new SpC to mg/L NaCl
C_salt_pulse = SpC_corr * k #this is from our calibration above!  


In [ ]:
#Define mass of salt in mg
mrec_g = 400 #mass of salt in g
mg_per_g = 1000
mrec_mg = mg_per_g*mrec_g

# Define the time interval from your logger
time_interval = 1 #second. Check your data to see if this ia the same for you! 


To calculate discharge from the conductivity measurements, we use

$$Q = \frac{m_{\rm rec}}{\Sigma_i (C_{salt,i} dt)},$$

where $Q$ is the discharge, $C_{salt,i}$ is the timeseries of salt concentrations (C_salt_pulse in our code), and $dt$ is the time between consecutive conductivity measurements (time_interval).

In [ ]:
#Now calculate discharge

discharge = mrec_mg / (C_salt_pulse.sum() * time_interval)

print('Discharge:',discharge, 'L/s')

In [ ]:
# Repeat this process for the salt tracer discharge measurements you conducted



### Bucket and stopwatch measurements

In [ ]:
# Here is an example bucket and stopwatch dataset
Q_bucket = read_csv('example-datasheets/Bucket-discharge.csv')
Q_bucket

In [ ]:
# Calculate discharge for each pair of volumes and times using Q = V/T.
# Then average to get the final discharge estimate.
